In [1]:
%reload_ext autoreload
%autoreload 2
import os
import sys
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import plotly.graph_objects as go

module_path = os.path.abspath(os.path.join('./../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from src.multicovariate_models.GroupsModelWrapper import GroupsModelWrapper
from src.multicovariate_models.DataLoader import DataLoader

base_dir = '../../'
rnd_seed = 3

In [2]:
df = pd.read_csv(os.path.join(base_dir, "data/dataframes/simulations-step0.1.csv"))
df = df.drop(columns=["origin_file"]).drop_duplicates()
"""main_df = main_df[['Sensor1', 'Sensor2', 'Sensor3', 'Sensor4', 'J-Apollo', 'J-RN2',
       'J-RN1', 'node_with_leak']].round(3).drop_duplicates()"""
df['leak_amount'] = df['leak_amount'].str[:-3].astype(float)

# drop all the `dpressure` columns
display(df)

,Sensor1,Sensor2,Sensor3,Sensor4,J-Apollo,J-RN2,J-RN1,node_with_leak,leak_amount,Sensor1_dpressure,Sensor2_dpressure,Sensor3_dpressure,Sensor4_dpressure,J-Apollo_dpressure,J-RN2_dpressure,J-RN1_dpressure
0,14.8003,15.9386,17.9611,19.3182,15.4271,15.9507,19.8981,Node_275-A,0.50,0.0134,0.0153,0.0082,0.0031,0.0,0.0006,0.0
1,14.7989,15.9371,17.9603,19.3179,15.4271,15.9507,19.8981,Node_275-A,0.55,0.0148,0.0168,0.0090,0.0034,0.0,0.0007,0.0
2,14.7976,15.9357,17.9595,19.3175,15.4271,15.9506,19.8981,Node_275-A,0.60,0.0162,0.0182,0.0098,0.0038,0.0,0.0007,0.0
3,14.7962,15.9342,17.9587,19.3172,15.4271,15.9505,19.8981,Node_275-A,0.65,0.0175,0.0197,0.0106,0.0041,0.0,0.0008,0.0
4,14.8045,15.9459,17.9640,19.3176,15.4271,15.9506,19.8981,Node_258-A,0.50,0.0093,0.0080,0.0053,0.0037,0.0,0.0007,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71994,14.7454,15.8914,17.9136,18.9397,15.4271,15.9221,19.8981,Node_Jonctiune-1225,10.00,0.0684,0.0625,0.0557,0.3816,0.0,0.0292,0.0
71995,14.7450,15.8911,17.9133,18.9370,15.4271,15.9219,19.8981,Node_Jonctiune-1225,10.05,0.0688,0.0628,0.0561,0.3843,0.0,0.0294,0.0
71997,14.5343,15.5478,17.6992,19.2515,15.4271,15.9396,19.8981,Node_Jonctiune-267,9.95,0.2795,0.4061,0.2701,0.0697,0.0,0.0117,0.0
71998,14.5326,15.5450,17.6974,19.2512,15.4271,15.9396,19.8981,Node_Jonctiune-267,10.00,0.2811,0.4089,0.2719,0.0701,0.0,0.0117,0.0


In [3]:
# create a node-leak response model
node_id_vec = df.node_with_leak.unique()

node_response_model_map = {}

for node_id in node_id_vec:
    df_node = df[df.node_with_leak == node_id].sort_values(by='leak_amount')
    
    leak_amount_vec = df_node.leak_amount
    sensor_dpressure_response_mat = df_node[DataLoader.SENSOR_DPRESSURE_COLS].to_numpy(dtype=float)
    
    node_response_model_map[node_id] = {
        'leak_amount_vec': leak_amount_vec,
        'sensor_dpressure_mat': sensor_dpressure_response_mat
    }

In [91]:
# construct the reference baseline profiles

fname_in = os.path.join(base_dir, 'data/sim-single-leak/base-pressure.csv')

base_pressure_df = pd.read_csv(fname_in)
base_sensor_pressure_df = base_pressure_df[DataLoader.SENSOR_COLS]

pressure_profile_sample_mat = base_sensor_pressure_df.sample(n=10, axis=0, random_state=rnd_seed).to_numpy(dtype=float)

n_cols = pressure_profile_sample_mat.shape[1]
n_rows = 4 + pressure_profile_sample_mat.shape[0]

ref_pressure_profile_mat = np.empty((n_rows, n_cols))
ref_pressure_profile_mat[0, :] = base_sensor_pressure_df.max().to_numpy(dtype=float)
ref_pressure_profile_mat[1, :] = base_sensor_pressure_df.min().to_numpy(dtype=float)
ref_pressure_profile_mat[2, :] = base_sensor_pressure_df.mean().to_numpy(dtype=float)
ref_pressure_profile_mat[3, :] = base_sensor_pressure_df.median().to_numpy(dtype=float)
ref_pressure_profile_mat[4:n_rows, :] = pressure_profile_sample_mat

In [92]:
# train-test split

data_loader = DataLoader(df)
X_train, X_test, y_train, y_test, enc_node_dict = data_loader.get_random_data_split_by_node()

Train data shape: (58608, 17), test data shape: (6624, 17)
x_train shape: (58608, 7), y_train shape: (58608,)


In [94]:
%%time

# create a feature matrix to train the model

import scipy.sparse
import random as rand

import src.analytics.linalg as linalg

n_rows = X_train.shape[0]
n_cols = X_train.shape[1]
n_profiles = ref_pressure_profile_mat.shape[0]
n_nodes = len(node_id_vec)
n_instances = n_rows


# calculate the difference between the reference profiles and measured profiles

# create a matrix of the form [x1; x1; x1; x1; x2; x2; x2; x2; ...]
sp_data = []
sp_rowidxs = []
sp_colidxs = []
for instanceN in range(n_rows):
    for profileN in range(n_profiles):
        colN = instanceN
        rowN = instanceN*n_profiles + profileN
        sp_data.append(1)
        sp_rowidxs.append(rowN)
        sp_colidxs.append(colN)
        
X_dupl_mat = scipy.sparse.csr_matrix((sp_data, (sp_rowidxs, sp_colidxs)), shape=(n_profiles*n_rows, n_rows))
X_train_dupl = X_dupl_mat.dot(X_train)

del X_dupl_mat

# create a matrix of the form [p1; p2; p3; p4; p1; p2; p3; p4; ...]
sp_data = []
sp_rowidxs = []
sp_colidxs = []
for instanceN in range(n_rows):
    for profileN in range(n_profiles):
        rowN = instanceN*n_profiles + profileN
        colN = profileN
        sp_data.append(1)
        sp_rowidxs.append(rowN)
        sp_colidxs.append(colN)

P_dupl_mat = scipy.sparse.csr_matrix((sp_data, (sp_rowidxs, sp_colidxs)), shape=(n_profiles*n_rows, n_profiles))
P_dupl = P_dupl_mat.dot(ref_pressure_profile_mat)

del sp_data
del sp_rowidxs
del sp_colidxs

# calculate the difference between all combinations of x_i, p_i
P_minus_X = P_dupl - X_train_dupl

del P_dupl
del X_train_dupl

print_count = 0

n_neg_per_pos = 20

new_cols = n_profiles*n_cols
new_rows = n_instances*(1 + n_neg_per_pos)

ftrvv = np.empty((new_rows, new_cols))
y = np.empty(new_rows)

ones_n_profiles = np.ones(n_profiles)
ones_d = np.ones(n_cols)

for instanceN in range(n_rows):
    P_minus_Xi = P_minus_X[(instanceN*n_profiles):((instanceN+1)*n_profiles), :]
    yi = y_train[instanceN]
    
    # generate a positive example
    node_id = data_loader.decode_node_label([yi])[0]
    #print(node_id)
    
    if instanceN % 1000 == 0:
        print(f'instance [{instanceN+1}/{n_rows}], node_id: {node_id}')
    
    node_response_model = node_response_model_map[node_id]
    leak_amount_vec = node_response_model['leak_amount_vec']
    sensor_dpressure_mat = node_response_model['sensor_dpressure_mat']
    
    n_leaks = len(leak_amount_vec)
    
    sub_all_comb = linalg.sub_all_comb(P_minus_Xi, sensor_dpressure_mat)
    measured_minus_expected_response_abs_diff_vec = np.dot(np.abs(sub_all_comb), ones_d)
    
    # find the best-matching expected response for each of the reference profiles
    for profileN in range(n_profiles):
        profile_measured_minus_expected_response_abs_diff_vec = measured_minus_expected_response_abs_diff_vec[profileN*n_leaks:(profileN+1)*n_leaks]
        best_matchN = np.argmin(profile_measured_minus_expected_response_abs_diff_vec)
        
        ref_profile_ftrvec = sub_all_comb[profileN*n_leaks + best_matchN]
        ftrvv[instanceN, profileN*n_cols:(profileN+1)*n_cols] = ref_profile_ftrvec
    
    y[instanceN] = 1
    
    # generate a negative example
    for negN in range(n_neg_per_pos):
        # select the node ID
        neg_node_id = node_id
        while neg_node_id == node_id:
            neg_node_id = rand.choice(node_id_vec)
        
        #print(neg_node_id)
        node_response_model = node_response_model_map[node_id]
        leak_amount_vec = node_response_model['leak_amount_vec']
        sensor_dpressure_mat = node_response_model['sensor_dpressure_mat']

        n_leaks = len(leak_amount_vec)

        sub_all_comb = linalg.sub_all_comb(P_minus_Xi, sensor_dpressure_mat)
        measured_minus_expected_response_abs_diff_vec = np.dot(np.abs(sub_all_comb), ones_d)
        
        rowN = n_instances + instanceN*n_neg_per_pos + negN

        for profileN in range(n_profiles):
            profile_measured_minus_expected_response_abs_diff_vec = measured_minus_expected_response_abs_diff_vec[profileN*n_leaks:(profileN+1)*n_leaks]
            best_matchN = np.argmin(profile_measured_minus_expected_response_abs_diff_vec)

            ref_profile_ftrvec = sub_all_comb[profileN*n_leaks + best_matchN]
            
            ftrvv[rowN, profileN*n_cols:(profileN+1)*n_cols] = ref_profile_ftrvec

        y[rowN] = 0

#print(ftrvv[1:10, :])
#print(y)

instance [1/58608], node_id: Node_-
instance [1001/58608], node_id: Node_258-A
instance [2001/58608], node_id: Node_J-1640
instance [3001/58608], node_id: Node_J-RN2
instance [4001/58608], node_id: Node_Jonctiune-1226
instance [5001/58608], node_id: Node_Jonctiune-1405
instance [6001/58608], node_id: Node_Jonctiune-1407
instance [7001/58608], node_id: Node_Jonctiune-1415
instance [8001/58608], node_id: Node_Jonctiune-1421
instance [9001/58608], node_id: Node_Jonctiune-1628
instance [10001/58608], node_id: Node_Jonctiune-1634
instance [11001/58608], node_id: Node_Jonctiune-1637
instance [12001/58608], node_id: Node_Jonctiune-1641
instance [13001/58608], node_id: Node_Jonctiune-1646
instance [14001/58608], node_id: Node_Jonctiune-1875
instance [15001/58608], node_id: Node_Jonctiune-1995
instance [16001/58608], node_id: Node_Jonctiune-1998
instance [17001/58608], node_id: Node_Jonctiune-2177
instance [18001/58608], node_id: Node_Jonctiune-2182
instance [19001/58608], node_id: Node_Jonctiu

In [48]:
ftrvv[n_rows - 5:n_rows + 20, 0:30]

array([[ 1.62834000e+00,  1.61547000e+00,  1.57041300e+00,
         1.26494400e+00,  1.37922000e+00,  1.51712500e+00,
         1.43015900e+00, -1.72515800e+00, -1.74245300e+00,
        -1.74069400e+00, -4.61278000e-01, -1.82735500e+00,
        -1.82490700e+00, -1.82731400e+00, -3.52656708e-01,
        -3.60892792e-01, -3.80780458e-01,  1.84329167e-03,
        -4.72527417e-01, -4.18065208e-01, -4.60263375e-01,
        -6.13004000e-01, -6.21455000e-01, -6.70370000e-01,
        -8.73500000e-04, -8.23864500e-01, -6.72576000e-01,
        -8.23826500e-01, -6.73587000e-01, -6.86102000e-01],
       [ 1.51534000e+00,  1.51297000e+00,  1.48301300e+00,
         1.09144000e-01,  1.37922000e+00,  1.46342500e+00,
         1.43015900e+00, -1.83815800e+00, -1.84495300e+00,
        -1.82809400e+00, -1.61707800e+00, -1.82735500e+00,
        -1.87860700e+00, -1.82731400e+00, -3.42956708e-01,
        -3.52192792e-01, -3.73380458e-01, -1.45356708e-01,
        -4.72527417e-01, -4.16365208e-01, -4.60263375e-

In [95]:
%%time

import sklearn

model = sklearn.ensemble.RandomForestClassifier(n_jobs=24)
model.fit(ftrvv, y)

CPU times: user 43min 23s, sys: 20.9 s, total: 43min 44s
Wall time: 2min 42s


RandomForestClassifier(n_jobs=24)

In [80]:
train_prob_vec = model.predict_proba(ftrvv)[:, 1]
train_prob_vec

array([0.17622883, 0.16793074, 0.17333405, ..., 0.15978066, 0.15978066,
       0.15978066])

In [96]:
# evaluate the model on the test set

n_rows = X_test.shape[0]
n_cols = X_test.shape[1]
n_profiles = ref_pressure_profile_mat.shape[0]
n_nodes = len(node_id_vec)
n_instances = n_rows

# shuffle X_test
idxs = list(range(n_rows))
rand.shuffle(idxs)
X_test = X_test[idxs, :]
y_test = y_test[idxs]

del idxs

# calculate the difference between the reference profiles and measured profiles

# create a matrix of the form [x1; x1; x1; x1; x2; x2; x2; x2; ...]
sp_data = []
sp_rowidxs = []
sp_colidxs = []
for instanceN in range(n_rows):
    for profileN in range(n_profiles):
        colN = instanceN
        rowN = instanceN*n_profiles + profileN
        sp_data.append(1)
        sp_rowidxs.append(rowN)
        sp_colidxs.append(colN)
        
X_dupl_mat = scipy.sparse.csr_matrix((sp_data, (sp_rowidxs, sp_colidxs)), shape=(n_profiles*n_rows, n_rows))
X_test_dupl = X_dupl_mat.dot(X_test)

del X_dupl_mat

# create a matrix of the form [p1; p2; p3; p4; p1; p2; p3; p4; ...]
sp_data = []
sp_rowidxs = []
sp_colidxs = []
for instanceN in range(n_rows):
    for profileN in range(n_profiles):
        rowN = instanceN*n_profiles + profileN
        colN = profileN
        sp_data.append(1)
        sp_rowidxs.append(rowN)
        sp_colidxs.append(colN)

P_dupl_mat = scipy.sparse.csr_matrix((sp_data, (sp_rowidxs, sp_colidxs)), shape=(n_profiles*n_rows, n_profiles))
P_dupl = P_dupl_mat.dot(ref_pressure_profile_mat)

del sp_data
del sp_rowidxs
del sp_colidxs

# calculate the difference between all combinations of x_i, p_i
P_minus_X = P_dupl - X_test_dupl

def create_ftrvv(node_id_vec, node_response_model_map, P_minus_Xi, n_profiles):
    n_cols = P_minus_Xi.shape[1]
    n_rows = len(node_id_vec)
    
    ftrvv = np.empty((n_rows, n_profiles*n_cols))
    
    for nodeN, node_id in enumerate(node_id_vec):
        node_response_model = node_response_model_map[node_id]
        leak_amount_vec = node_response_model['leak_amount_vec']
        sensor_dpressure_mat = node_response_model['sensor_dpressure_mat']
        
        n_leaks = len(leak_amount_vec)
    
        sub_all_comb = linalg.sub_all_comb(P_minus_Xi, sensor_dpressure_mat)
        measured_minus_expected_response_abs_diff_vec = np.dot(np.abs(sub_all_comb), np.ones(n_cols))

        ftrv = np.empty(n_profiles*n_cols)

        for profileN in range(n_profiles):
            profile_measured_minus_expected_response_abs_diff_vec = measured_minus_expected_response_abs_diff_vec[profileN*n_leaks:(profileN+1)*n_leaks]
            best_matchN = np.argmin(profile_measured_minus_expected_response_abs_diff_vec)

            ref_profile_ftrvec = sub_all_comb[profileN*n_leaks + best_matchN]
            ftrv[profileN*n_cols:(profileN+1)*n_cols] = ref_profile_ftrvec
        
        ftrvv[nodeN, :] = ftrv
    
    return ftrvv

total_correct = 0

# for each instance go over all the nodes and select the one that the model is most confident in
y_hat = np.empty(n_rows)
for instanceN in range(n_rows):
    P_minus_Xi = P_minus_X[(instanceN*n_profiles):((instanceN+1)*n_profiles), :]
    yi = y_test[instanceN]
    
    node_model_confidence_vec = np.empty(len(node_id_vec))
    
    ftrvv_test = create_ftrvv(node_id_vec, node_response_model_map, P_minus_Xi, n_profiles)
        
    node_model_confidence_vec = model.predict_proba(ftrvv_test)
    node_model_confidence_vec = node_model_confidence_vec[:, 1]
    
    sort_idxs = np.argsort(-node_model_confidence_vec)
    
    best_idxs = sort_idxs[:5]
    
    best_node_ids = node_id_vec[best_idxs]
    
    real_node_id = data_loader.decode_node_label([yi])[0]
    print(f'real: {real_node_id}, best: {best_node_ids}')
    
    yi_hat_arr = data_loader.encode_node_id([val for val in best_node_ids])
    y_hat[instanceN] = yi_hat_arr[0]
    
    total_correct += yi in yi_hat_arr
    
    if instanceN % 10 == 0:
        print(f'instance [{instanceN+1}/{n_rows}], CA (top k): {total_correct / (instanceN + 1)}')

ca = total_correct / n_rows
print(f'CA: {ca}')
    

real: Node_Jonctiune-2205, best: ['Node_Jonctiune-3471' 'Node_Jonctiune-J-21' 'Node_Sensor4'
 'Node_Jonctiune-3067' 'Node_Jonctiune-267']
instance [1/6624], CA (top k): 0.0
real: Node_Sensor2, best: ['Node_PT1' 'Node_Jonctiune-1632' 'Node_Jonctiune-267' 'Node_258-A'
 'Node_Jonctiune-2737']
real: Node_Jonctiune-2185, best: ['Node_Jonctiune-3067' 'Node_Jonctiune-3386' 'Node_J-RN1' 'Node_J-Apollo'
 'Node_Jonctiune-2968']
real: Node_275-A, best: ['Node_Jonctiune-J-15' 'Node_Jonctiune-2735' 'Node_PT2'
 'Node_Jonctiune-1628' 'Node_Jonctiune-2744']
real: Node_Jonctiune-1634, best: ['Node_Jonctiune-3917' 'Node_Jonctiune-2180' 'Node_Jonctiune-3967'
 'Node_Sensor2' 'Node_Jonctiune-J-1']
real: Node_J-1640, best: ['Node_Sensor2' 'Node_Jonctiune-267' 'Node_Jonctiune-1413'
 'Node_Jonctiune-4731' 'Node_Jonctiune-2185']
real: Node_Jonctiune-2753, best: ['Node_275-A' 'Node_Jonctiune-267' 'Node_Jonctiune-1415'
 'Node_Jonctiune-1413' 'Node_Jonctiune-1996']
real: Node_Jonctiune-1634, best: ['Node_Jonctiun

real: Node_PT1, best: ['Node_Sensor4' 'Node_Jonctiune-1226' 'Node_Jonctiune-1225'
 'Node_Jonctiune-3466' 'Node_Jonctiune-267']
real: Node_Jonctiune-J-34, best: ['Node_Jonctiune-267' 'Node_Jonctiune-2180' 'Node_Jonctiune-3967'
 'Node_Jonctiune-1413' 'Node_Jonctiune-3471']
real: Node_Jonctiune-3913, best: ['Node_Jonctiune-3972' 'Node_PT2' 'Node_Jonctiune-1628'
 'Node_Jonctiune-2735' 'Node_Jonctiune-1995']
instance [61/6624], CA (top k): 0.0
real: Node_Jonctiune-3956, best: ['Node_Sensor4' 'Node_Jonctiune-3067' 'Node_Jonctiune-1225'
 'Node_Jonctiune-2187' 'Node_Jonctiune-2193']
real: Node_J-RN1, best: ['Node_Jonctiune-1406' 'Node_Jonctiune-1407' 'Node_Jonctiune-1635'
 'Node_Jonctiune-1874' 'Node_Jonctiune-3075']
real: Node_Jonctiune-1642, best: ['Node_Jonctiune-3917' 'Node_Jonctiune-2729' 'Node_Jonctiune-3425'
 'Node_Jonctiune-J-1' 'Node_Jonctiune-J-27']
real: Node_Jonctiune-2196, best: ['Node_Jonctiune-J-15' 'Node_Jonctiune-1634' 'Node_Jonctiune-2737'
 'Node_PT1' 'Node_Jonctiune-1632']
r

real: Node_Jonctiune-1226, best: ['Node_Jonctiune-3913' 'Node_Jonctiune-2879' 'Node_Jonctiune-2201'
 'Node_Jonctiune-1998' 'Node_Jonctiune-267']
real: Node_Jonctiune-2729, best: ['Node_Jonctiune-3967' 'Node_Jonctiune-2180' 'Node_Sensor2'
 'Node_Jonctiune-1636' 'Node_Jonctiune-1637']
real: Node_Jonctiune-3956, best: ['Node_Jonctiune-3471' 'Node_Jonctiune-4723' 'Node_Jonctiune-3422'
 'Node_Sensor4' 'Node_Jonctiune-2752']
real: Node_Jonctiune-12372, best: ['Node_Jonctiune-3067' 'Node_Jonctiune-2753' 'Node_Jonctiune-3917'
 'Node_Jonctiune-2752' 'Node_255-A']
real: Node_Jonctiune-3510, best: ['Node_Jonctiune-3913' 'Node_Jonctiune-2197' 'Node_Jonctiune-3917'
 'Node_Jonctiune-3067' 'Node_Jonctiune-1226']
instance [121/6624], CA (top k): 0.0
real: Node_Jonctiune-2777, best: ['Node_J-RN2' 'Node_Jonctiune-2730' 'Node_Jonctiune-3425'
 'Node_Jonctiune-J-1' 'Node_Jonctiune-J-25']
real: Node_Jonctiune-2204, best: ['Node_Jonctiune-2187' 'Node_Jonctiune-2192' 'Node_Jonctiune-1226'
 'Node_Jonctiune-391

real: Node_Jonctiune-2197, best: ['Node_Jonctiune-2879' 'Node_Jonctiune-3913' 'Node_Jonctiune-4595'
 'Node_Jonctiune-3464' 'Node_Jonctiune-1872']
real: Node_Jonctiune-1405, best: ['Node_Jonctiune-2197' 'Node_Jonctiune-J-21' 'Node_Jonctiune-1225'
 'Node_Jonctiune-1226' 'Node_Jonctiune-3448']
real: Node_Jonctiune-3075, best: ['Node_Jonctiune-2737' 'Node_Jonctiune-2180' 'Node_Jonctiune-2736'
 'Node_Jonctiune-1634' 'Node_Sensor2']
real: Node_Jonctiune-3566, best: ['Node_Jonctiune-2177' 'Node_Jonctiune-J-21' 'Node_Jonctiune-3917'
 'Node_Jonctiune-2186' 'Node_Jonctiune-2197']
real: Node_Jonctiune-3917, best: ['Node_Jonctiune-1874' 'Node_275-A' 'Node_Jonctiune-3972'
 'Node_Jonctiune-2190' 'Node_Jonctiune-2177']
real: Node_Jonctiune-12372, best: ['Node_Jonctiune-3067' 'Node_Jonctiune-3917' 'Node_255-A'
 'Node_Jonctiune-2752' 'Node_Sensor4']
real: Node_J-RN1, best: ['Node_Jonctiune-1406' 'Node_Jonctiune-1407' 'Node_Jonctiune-1635'
 'Node_Jonctiune-1874' 'Node_Jonctiune-3075']
real: Node_J-Apoll

real: Node_Jonctiune-2206, best: ['Node_Jonctiune-2730' 'Node_Jonctiune-2729' 'Node_Jonctiune-3425'
 'Node_Jonctiune-J-1' 'Node_Jonctiune-J-25']
real: Node_PT3, best: ['Node_Jonctiune-3067' 'Node_Jonctiune-J-26' 'Node_Jonctiune-2968'
 'Node_J-Apollo' 'Node_Jonctiune-3386']
real: Node_Jonctiune-2735, best: ['Node_Jonctiune-1635' 'Node_258-A' 'Node_Jonctiune-2204' 'Node_PT1'
 'Node_Jonctiune-1632']
real: Node_Jonctiune-2756, best: ['Node_Jonctiune-1415' 'Node_Jonctiune-2190' 'Node_Jonctiune-4731'
 'Node_Jonctiune-1405' 'Node_Jonctiune-2189']
real: Node_Jonctiune-J-23, best: ['Node_Jonctiune-3448' 'Node_Jonctiune-2729' 'Node_Jonctiune-2730'
 'Node_Jonctiune-J-1' 'Node_Jonctiune-J-27']
real: Node_Sensor4, best: ['Node_Jonctiune-2190' 'Node_275-A' 'Node_Jonctiune-4731'
 'Node_Jonctiune-12372' 'Node_Jonctiune-2185']
real: Node_Jonctiune-2203, best: ['Node_Jonctiune-2756' 'Node_Jonctiune-3446' 'Node_Jonctiune-3917'
 'Node_Jonctiune-J-1' 'Node_Jonctiune-J-27']
real: Node_Jonctiune-2195, best: 

...

real: Node_PT4, best: ['Node_Jonctiune-3967' 'Node_Jonctiune-2879' 'Node_Jonctiune-1995'
 'Node_Jonctiune-1996' 'Node_Jonctiune-1997']
real: Node_Jonctiune-4595, best: ['Node_Jonctiune-2203' 'Node_Jonctiune-2201' 'Node_Jonctiune-1874'
 'Node_Jonctiune-3067' 'Node_Jonctiune-1635']
real: Node_Jonctiune-J-21, best: ['Node_275-A' 'Node_Jonctiune-2190' 'Node_Jonctiune-2193'
 'Node_Jonctiune-2180' 'Node_Jonctiune-2187']
real: Node_Jonctiune-2777, best: ['Node_Jonctiune-3067' 'Node_Jonctiune-1226' 'Node_Jonctiune-1225'
 'Node_Jonctiune-J-19' 'Node_Sensor4']
real: Node_Sensor4, best: ['Node_275-A' 'Node_Jonctiune-2186' 'Node_Jonctiune-3471' 'Node_Sensor3'
 'Node_Jonctiune-3972']
real: Node_Jonctiune-2197, best: ['Node_Jonctiune-3956' 'Node_Jonctiune-2202' 'Node_Jonctiune-J-34'
 'Node_Jonctiune-3972' 'Node_PT1']
real: Node_275-A, best: ['Node_Jonctiune-3448' 'Node_Jonctiune-J-1' 'Node_Jonctiune-3425'
 'Node_Jonctiune-2729' 'Node_Jonctiune-2730']
real: Node_Jonctiune-J-27, best: ['Node_Jonctiune

real: Node_Jonctiune-1414, best: ['Node_255-A' 'Node_Sensor4' 'Node_Jonctiune-J-21' 'Node_Jonctiune-1225'
 'Node_Jonctiune-1226']
real: Node_Jonctiune-2729, best: ['Node_Jonctiune-1610' 'Node_Sensor2' 'Node_Jonctiune-4723'
 'Node_Jonctiune-3422' 'Node_Jonctiune-2187']
real: Node_Jonctiune-2201, best: ['Node_Jonctiune-J-15' 'Node_Jonctiune-3967' 'Node_Jonctiune-1634'
 'Node_Jonctiune-12372' 'Node_Jonctiune-2185']
real: Node_Jonctiune-1419, best: ['Node_Jonctiune-2203' 'Node_Jonctiune-3074' 'Node_Jonctiune-2201'
 'Node_Jonctiune-2739' 'Node_Jonctiune-2207']
real: Node_Jonctiune-2776, best: ['Node_Jonctiune-3067' 'Node_Jonctiune-2753' 'Node_J-RN2'
 'Node_Jonctiune-1226' 'Node_Jonctiune-J-1']
real: Node_Jonctiune-2197, best: ['Node_Jonctiune-2180' 'Node_Jonctiune-1610' 'Node_Jonctiune-J-23'
 'Node_Jonctiune-2201' 'Node_Jonctiune-2203']
real: Node_Jonctiune-3422, best: ['Node_Sensor3' 'Node_Jonctiune-2879' 'Node_Sensor2' 'Node_Jonctiune-4615'
 'Node_Jonctiune-3967']
real: Node_Jonctiune-307

real: Node_Jonctiune-2186, best: ['Node_Jonctiune-2187' 'Node_Jonctiune-2750' 'Node_Jonctiune-2180'
 'Node_Jonctiune-3467' 'Node_J-RN2']
real: Node_Jonctiune-2201, best: ['Node_Jonctiune-4731' 'Node_Jonctiune-3067' 'Node_Jonctiune-1610'
 'Node_Jonctiune-2182' 'Node_Jonctiune-2751']
instance [2891/6624], CA (top k): 0.0
real: Node_Jonctiune-2201, best: ['Node_J-RN2' 'Node_Jonctiune-4615' 'Node_Jonctiune-2751'
 'Node_Jonctiune-3067' 'Node_Jonctiune-3074']
real: Node_Jonctiune-2737, best: ['Node_Jonctiune-3917' 'Node_Jonctiune-267' 'Node_Jonctiune-3466'
 'Node_Sensor4' 'Node_Jonctiune-2197']
real: Node_Jonctiune-2202, best: ['Node_Sensor2' 'Node_Jonctiune-267' 'Node_Jonctiune-1413'
 'Node_Jonctiune-2180' 'Node_Jonctiune-3967']
real: Node_Jonctiune-2729, best: ['Node_Jonctiune-2204' 'Node_Jonctiune-2203' 'Node_Jonctiune-2201'
 'Node_Jonctiune-2180' 'Node_258-A']
real: Node_Jonctiune-3425, best: ['Node_Jonctiune-3967' 'Node_Sensor2' 'Node_Jonctiune-2181'
 'Node_Jonctiune-2180' 'Node_275-A']

real: Node_Jonctiune-1616, best: ['Node_Jonctiune-2197' 'Node_Jonctiune-2729' 'Node_Jonctiune-J-25'
 'Node_Jonctiune-2730' 'Node_Jonctiune-J-27']
real: Node_Jonctiune-1995, best: ['Node_Sensor4' 'Node_Jonctiune-3446' 'Node_Jonctiune-2756'
 'Node_Jonctiune-1225' 'Node_Jonctiune-3067']
real: Node_J-Apollo, best: ['Node_Jonctiune-1406' 'Node_Jonctiune-1407' 'Node_Jonctiune-1635'
 'Node_Jonctiune-1874' 'Node_Jonctiune-3075']
real: Node_Jonctiune-1874, best: ['Node_Sensor2' 'Node_Jonctiune-2180' 'Node_Jonctiune-3967'
 'Node_Jonctiune-2744' 'Node_Jonctiune-1642']
instance [2951/6624], CA (top k): 0.0
real: Node_Sensor3, best: ['Node_Jonctiune-3448' 'Node_Jonctiune-J-1' 'Node_Jonctiune-3425'
 'Node_Jonctiune-J-27' 'Node_Jonctiune-2730']
real: Node_Jonctiune-3386, best: ['Node_Jonctiune-1406' 'Node_Jonctiune-1407' 'Node_Jonctiune-1635'
 'Node_Jonctiune-1874' 'Node_Jonctiune-3075']
real: Node_Jonctiune-2735, best: ['Node_Jonctiune-3448' 'Node_Sensor4' 'Node_Sensor2' 'Node_Jonctiune-3067'
 'Node

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [90]:
1 in [4, 2, 4, 1]

True